In [6]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.requests import Request
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import time
from uuid import uuid4
from fastapi.staticfiles import StaticFiles
from API.model.preprocessing import data_load
from API.model.model_training import model_training
import re
import sqlite3


In [ ]:
def sequence_mining(team, opponent, df, curr_team, season):
    combined_df = df.replace({str(team): 'same', str(opponent): 'other'}, regex=True)
    
    transformed_data = {}
    encoders = []
    for column in combined_df.columns[:-1]:
        le = LabelEncoder()
        encoders.append(le)
        transformed_data[column] = le.fit_transform(combined_df[column])
    
    transformed_df = pd.DataFrame(transformed_data)
    df = pd.concat([transformed_df, combined_df[combined_df.columns[-1]]], axis=1)
    
    undersample_len = len(df[df['class'] == 1])
    undersample_df = df[df['class'] == 0].sample(n=undersample_len, random_state=43)
    df = pd.concat([df[df['class'] == 1], undersample_df])
    
    events_idx = {}
    total_sequences = len(df[df['class'] == 1])
    freqs = []
    
    for j, event in zip(range(12, 112, 11), range(10, 0, -1)):
        event_len_df = df[df['class'] == 1].iloc[:, -j:-1] 
        total_sequences_same_length = len(event_len_df)
        
        event_dict = {}
        row_counts = defaultdict(int)
        
        for i in range(len(event_len_df)):
            row_tuple = tuple(event_len_df.iloc[i])
            row_counts[row_tuple] += 1
            
        
        sorted_row_counts = sorted(row_counts.items(), key=lambda x: x[1], reverse=True)
        
        total_seqs = 0
        for i in range(10):
            total_seqs += sorted_row_counts[i][1]

        
        if sorted_row_counts:
            mc_row, max_count = sorted_row_counts[0]
            sc_row, second_max_count = sorted_row_counts[1] if len(sorted_row_counts) > 1 else (None, 0)
        else:
            mc_row, max_count, sc_row, second_max_count = None, 0, None, 0
        
        max_count_ratio = max_count / total_seqs
        second_max_count_ratio = second_max_count / total_seqs
        
        mc_indices = event_len_df.apply(lambda row: tuple(row) == mc_row, axis=1)
        mc_indices = mc_indices[mc_indices].index.tolist()
        
        sc_indices = event_len_df.apply(lambda row: tuple(row) == sc_row, axis=1)
        sc_indices = sc_indices[sc_indices].index.tolist()
        
        events_idx[event] = mc_indices
        
        event_dict['Event'] = abs(event - 11)
        event_dict['Frequency'] = max_count
        event_dict['Ratio'] = np.round(max_count_ratio, 4)
        event_dict['Sec Frequency'] = second_max_count
        event_dict['Sec Ratio'] = np.round(second_max_count_ratio, 4)
        
        try:
            event_dict['Sequence'] = str(combined_df.iloc[mc_indices[0], -j:-1].to_frame().dropna().T.to_dict(orient="records"))
        except IndexError:
            print(mc_indices, j, curr_team, season)

        try:
            event_dict['Sec Sequence'] = str(combined_df.iloc[sc_indices[0], -j:-1].to_frame().dropna().T.to_dict(orient="records"))
        except IndexError:
            print(sc_indices, j, curr_team, season)
        
        freqs.append(event_dict)
    
    return pd.DataFrame(freqs)


In [ ]:
def analyze_team(team, season, venue):
    og_df = pd.read_csv('data/'+season)

    def team_selection(pref_team, df):
        if pref_team in df.HomeTeam.unique():
            pref_df = df[df.HomeTeam == pref_team]
            return pref_df
        else:
            return None
        

    new_df = team_selection(team, og_df)

    if new_df is None:
        print(f"No data found for team {team} in {season}")
        return [] 

    factors = ['ShotDist','TimeoutTeam','Substitution', 'Shooter',
               'Rebounder', 'Blocker','Fouler',
               'ReboundType','ViolationPlayer',
               'FreeThrowShooter','TurnoverPlayer']

    fact_cols = [col + str((i // 11) % 10 + 1) for i, col in enumerate(factors * 10)]
    fact_cols.append('class')

    new_df['ShotDist'] = new_df.ShotDist.apply(lambda x: 'close' if x <= 10 else '3pt' if x >= 22 else 'mid' if pd.notna(x) else x)
    
    new_df['TimeoutTeam'] = new_df.apply(
        lambda row: 'timeout_home' if pd.notna(row['TimeoutTeam']) and row['TimeoutTeam'] == row['HomeTeam'] 
        else 'timeout_away' if pd.notna(row['TimeoutTeam']) 
        else row['TimeoutTeam'], 
        axis=1
    )

    new_df['Shooter'] = new_df.apply(lambda row: 'shooter_home' if pd.notna(row['Shooter']) and pd.notna(row['HomePlay'])
                                         else 'shooter_away' if pd.notna(row['Shooter']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['Rebounder'] = new_df.apply(lambda row: 'rebounder_home' if pd.notna(row['Rebounder']) and pd.notna(row['HomePlay'])
                                         else 'rebounder_away' if pd.notna(row['Rebounder']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['Blocker'] = new_df.apply(lambda row: 'blocker_home' if pd.notna(row['Blocker']) and pd.notna(row['HomePlay'])
                                         else 'blocker_away' if pd.notna(row['Blocker']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['Fouler'] = new_df.apply(lambda row: 'fouler_home' if pd.notna(row['Fouler']) and pd.notna(row['HomePlay'])
                                         else 'fouler_away' if pd.notna(row['Fouler']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['ViolationPlayer'] = new_df.apply(lambda row: 'violator_home' if pd.notna(row['ViolationPlayer']) and pd.notna(row['HomePlay'])
                                         else 'violator_away' if pd.notna(row['ViolationPlayer']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['FreeThrowShooter'] = new_df.apply(lambda row: 'ft_home' if pd.notna(row['FreeThrowShooter']) and pd.notna(row['HomePlay'])
                                         else 'ft_away' if pd.notna(row['FreeThrowShooter']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['TurnoverPlayer'] = new_df.apply(lambda row: 'to_player_home' if pd.notna(row['TurnoverPlayer']) and pd.notna(row['HomePlay'])
                                         else 'to_player_away' if pd.notna(row['TurnoverPlayer']) and pd.notna(row['AwayPlay'])
                                         else np.nan,
                                         axis=1)

    new_df['Substitution'] = new_df.apply(lambda row: 'sub_home' if pd.notna(row['EnterGame']) and pd.notna(row['HomePlay'])
                                                  else 'sub_away' if pd.notna(row['EnterGame']) and pd.notna(row['AwayPlay'])
                                                  else np.nan,
                                                  axis=1)

    def home_runner(data):
        home_runs = []
        current_run = []
        for idx in data.index:
            home_made_shot = pd.notna(data.at[idx, 'HomePlay']) and 'makes' in data.at[idx, 'HomePlay']
            away_made_shot = pd.notna(data.at[idx, 'AwayPlay']) and 'makes' in data.at[idx, 'AwayPlay']

            if home_made_shot:
                current_run.append(idx)
            elif away_made_shot:
                if len(current_run) >= 4:
                    home_runs.append(current_run.copy())
                current_run.clear()
                
        if len(current_run) >= 4:
            home_runs.append(current_run.copy())
            
        return home_runs

    def away_runner(data):
        away_runs = []
        current_run = []
        for idx in data.index:
            away_made_shot = pd.notna(data.at[idx, 'AwayPlay']) and 'makes' in data.at[idx, 'AwayPlay']
            home_made_shot = pd.notna(data.at[idx, 'HomePlay']) and 'makes' in data.at[idx, 'HomePlay']

            if away_made_shot:
                current_run.append(idx)
            elif home_made_shot:
                if len(current_run) >= 4:
                    away_runs.append(current_run.copy())
                current_run.clear()

        if len(current_run) >= 4:
            away_runs.append(current_run.copy())
            
        return away_runs

    home_runs = home_runner(new_df)
    away_runs = away_runner(new_df)

    all_runs = []
    all_runs.extend(home_runs)
    all_runs.extend(away_runs)

    new_df = new_df[factors]

    def runs_iter(data, runs):
        runs_df = pd.DataFrame()
        for run in runs:
            if run[0]-10 in data.index and run[0]-1 in data.index:
                a = data.loc[run[0]-10:run[0]-1, factors].values.ravel()
                a = np.append(a,1)
                runs_df = pd.concat([runs_df,pd.DataFrame([a.copy()])])
        return runs_df

    venue_runs = home_runs if venue == 'home' else away_runs
    
    runs_df = runs_iter(new_df, venue_runs)
    
    if not runs_df.empty:
        runs_df.columns = fact_cols
        runs_df['class'] = runs_df['class'].fillna(1)
    else:
        runs_df = pd.DataFrame(columns=fact_cols)


    def no_runs_preprocessing(data, runs):
        r = [i[0] for i in runs]  
        r_x = []
        for num in r:
            r_x.extend(range(num - 10, num + 1))

        no_runs_df = data[~data.index.isin(r_x)].reset_index(drop=True)

        segment_size = 10
        segments = len(no_runs_df) // segment_size

        if segments > 0:
            no_runs_split = np.array_split(no_runs_df, segments)
        else:
            no_runs_split = []

        no_runs_split = [x for x in no_runs_split if len(x) == 10]

        return no_runs_split

    def no_runs_optimized(data, factors, fact_cols):
        if not data: 
            return pd.DataFrame(columns=fact_cols) 
            
        no_runs_df = pd.DataFrame([np.append(segment.loc[:, factors].values.ravel(), int(0)) for segment in data])
        no_runs_df.columns = fact_cols
        return no_runs_df

    processed_no_runs = no_runs_preprocessing(new_df, venue_runs)
    no_runs_df = no_runs_optimized(processed_no_runs, factors, fact_cols)

    combined_df = pd.concat([runs_df,no_runs_df],ignore_index=True)
    
    if combined_df.empty:
        print(f"No run or no-run sequences found for {team} {season} {venue}")
        return [] 

    os.makedirs('team_runs', exist_ok=True)
    combined_df.to_csv('team_runs/'+str(team)+'_'+str(venue)+'_runs.csv', index=False)
    combined_df = pd.read_csv('team_runs/'+str(team)+'_'+str(venue)+'_runs.csv')


    if venue == 'home':
        df_data = sequence_mining('home', 'away',combined_df, team, season)
    elif venue == 'away':
         df_data = sequence_mining('away', 'home' ,combined_df, team, season)

    df_data = df_data.to_dict(orient="records")

    return df_data

In [21]:
teams = ['DET', 'CLE', 'NOP', 'WAS', 'PHI', 'CHI', 'UTA', 'CHO', 'IND',
       'DEN', 'NYK', 'SAS', 'DAL', 'LAC', 'MIN', 'MEM', 'ATL', 'MIA',
       'OKC', 'TOR', 'BRK', 'GSW', 'LAL', 'POR', 'PHO', 'SAC', 'HOU',
       'MIL', 'ORL', 'BOS']

seasons = ['NBA_PBP_2015-16.csv', 
        'NBA_PBP_2016-17.csv',
        'NBA_PBP_2017-18.csv',
        'NBA_PBP_2018-19.csv',
        'NBA_PBP_2019-20.csv',
        ]

venues = ['home', 'away']

In [ ]:
all_dfs = []

for team in teams:
    for season in seasons:
        for venue in venues:
            try:
                team_mining_list = analyze_team(team, season, venue)
                
                if team_mining_list:
                    for record in team_mining_list:
                        record['team'] = team
                        record['season'] = re.sub(r'[^0-9-]+','',season)
                        record['venue'] = venue
                    
                    team_df = pd.DataFrame(team_mining_list)
                    all_dfs.append(team_df)
                
            except IndexError:
                print(f"IndexError processing {team} {season} {venue}")
            except Exception as e:
                print(f"Error processing {team} {season} {venue}: {e}")

if all_dfs:
    all_data = pd.concat(all_dfs, ignore_index=True)
    print("All data processed and concatenated.")
else:
    print("No data was processed.")
    all_data = pd.DataFrame()

All data processed and concatenated.


In [24]:

def initialize_db():
    conn = sqlite3.connect('frequencies_new.db')
    cursor = conn.cursor()
    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS freqs(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                event TEXT,
                frequency INTEGER,
                ratio REAL,
                sec_frequency INTEGER,
                sec_ratio REAL,
                team TEXT,
                season TEXT,
                venue TEXT,
                sequence TEXT,
                sec_sequence TEXT
                )
                '''
                )
    conn.commit()
    conn.close()

initialize_db()

In [25]:
all_data_dict = all_data.to_dict(orient='records')

records_to_insert = [
    (
        record.get('Event'),
        record.get('Frequency'),
        record.get('Ratio'),
        record.get('Sec Frequency'),
        record.get('Sec Ratio'),
        record.get('team'),
        record.get('season'),
        record.get('venue'),
        record.get('Sequence'),
        record.get('Sec Sequence')
    )
    for record in all_data_dict
]

In [26]:
conn = sqlite3.connect('frequencies_new.db')
cursor = conn.cursor()

cursor.executemany('''
                    INSERT INTO freqs (
                   event, frequency, ratio, sec_frequency, sec_ratio, team, season, venue, sequence, sec_sequence)
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                   ''', records_to_insert)

conn.commit()
conn.close()

In [27]:
conn = sqlite3.connect('frequencies_new.db')
cursor = conn.cursor()

cursor.execute('''
                SELECT sequence FROM freqs
               ''')

seqs = cursor.fetchall()

cursor.close()
conn.close()

seqs[0]


("[{'Fouler10': 'fouler_same'}]",)

In [28]:
import sqlite3
import re

def browse_patterns(team, season, db):
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    cursor.execute('''
                SELECT 
        event,
        SUM(frequency) AS total_frequency,
        AVG(ratio) AS total_ratio,
        sequence,
        SUM(sec_frequency) AS total_sec_frequency,
        AVG(sec_ratio) as total_sec_ratio,
        sec_sequence,
        season,
        venue
        
    FROM 
        freqs
    WHERE team = ? and event != '10' and season = ?
    GROUP BY 
        event
    ORDER BY 
        event ASC
    LIMIT 5

                ''', (team, re.sub(r'NBA_PBP_|.csv','',season)))
    

    rows = cursor.fetchall()
    

    df_rows = pd.DataFrame(rows)
    df_rows.columns = ['event','total_frequency','total_ratio', 'sequence', 
                    'total_sec_frequency', 'total_sec_ratio', 'sec_sequence', 'season', 'venue']


    cursor.close()
    conn.close()
    
    return df_rows

In [ ]:
import sqlite3
import pandas as pd
import re
import os
from openpyxl import load_workbook, Workbook

def browse_patterns(teams, seasons, db, output_file):
    """
    Creates an Excel file with separate sheets for each team.
    If a team's sheet exists, it appends new season data to it.
    
    Args:
        teams (list): List of team names.
        seasons (list): List of seasons to query.
        db (str): Path to the SQLite database.
        output_file (str): Output Excel file name.
    """
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    if not os.path.exists(output_file) or os.stat(output_file).st_size == 0:
        print(f'Creating new file: {output_file}')
        book = Workbook()
        book.save(output_file)

    book = load_workbook(output_file)
        
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        writer._book = book  

        for team in teams:
            for season in seasons:
                clean_season = re.sub(r'NBA_PBP_|.csv', '', season)
                
                cursor.execute('''
                    SELECT 
                        event,
                        SUM(frequency) AS total_frequency,
                        AVG(ratio) AS total_ratio,
                        sequence,
                        SUM(sec_frequency) AS total_sec_frequency,
                        AVG(sec_ratio) as total_sec_ratio,
                        sec_sequence,
                        season,
                        venue
                    FROM 
                        freqs
                    WHERE team = ? AND event != '10' AND season = ?
                    GROUP BY 
                        event, venue
                    ORDER BY 
                        event ASC
                    LIMIT 10
                ''', (team, clean_season))
                
                rows = cursor.fetchall()
                df_rows = pd.DataFrame(rows, columns=['event','total_frequency','total_ratio', 'sequence', 
                    'total_sec_frequency', 'total_sec_ratio', 'sec_sequence', 'season', 'venue'])
                
                if df_rows.empty:
                    print(f"No data found for {team} {clean_season}")
                    continue

                if team in book.sheetnames:
                    startrow = book[team].max_row
                    df_rows.to_excel(writer, sheet_name=team, index=False, startrow=startrow, header=False)
                else:
                    df_rows.to_excel(writer, sheet_name=team, index=False)

                print(f"Added season {clean_season} data for team {team} to {output_file}")

    try:
        book = load_workbook(output_file)
        if "Sheet" in book.sheetnames and len(book.sheetnames) > 1:
            book.remove(book["Sheet"])
            book.save(output_file)
            print("Removed default 'Sheet' from the final file.")
    except Exception as e:
        print(f"Could not remove default 'Sheet': {e}")
        
    cursor.close()
    conn.close()

In [36]:
browse_patterns(teams, seasons, 'frequencies_new.db','queries.xlsx')

Added season 2015-16 data for team DET to queries.xlsx
Added season 2016-17 data for team DET to queries.xlsx
Added season 2017-18 data for team DET to queries.xlsx
Added season 2018-19 data for team DET to queries.xlsx
Added season 2019-20 data for team DET to queries.xlsx
Added season 2015-16 data for team CLE to queries.xlsx
Added season 2016-17 data for team CLE to queries.xlsx
Added season 2017-18 data for team CLE to queries.xlsx
Added season 2018-19 data for team CLE to queries.xlsx
Added season 2019-20 data for team CLE to queries.xlsx
Added season 2015-16 data for team NOP to queries.xlsx
Added season 2016-17 data for team NOP to queries.xlsx
Added season 2017-18 data for team NOP to queries.xlsx
Added season 2018-19 data for team NOP to queries.xlsx
Added season 2019-20 data for team NOP to queries.xlsx
Added season 2015-16 data for team WAS to queries.xlsx
Added season 2016-17 data for team WAS to queries.xlsx
Added season 2017-18 data for team WAS to queries.xlsx
Added seas